In [ ]:
import time
import numpy as np
import nmrglue as ng
import MRAi_2D as ma
import Proc_2D as pr
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size':25})

In [ ]:
%%time
spectrum = "Malt.ft2"
dic , Ref = ng.pipe.read("Spectra/"+spectrum)
Echo = pr.convertors.VE_to_E(Ref)
Anti = pr.convertors.VE_to_AE(Ref)

In [ ]:
%%time
DNN_dirc = "Trained_NN/Echo/"

WNN_Echo = ma.Production.Echo_Rec(Echo,DNN_dirc+"EAE/")
WNN_Anti = pr.convertors.flip(ma.Production.Echo_Rec(pr.convertors.flip(Anti),DNN_dirc+"EAE/"))

In [ ]:
%%time
CS_Echo = pr.CS.E_US(Echo,200)
CS_Anti = pr.convertors.flip(pr.CS.E_US(pr.convertors.flip(Anti),200))

In [ ]:
%%time
np.random.seed(0)
ind_NUS = pr.NUS_sampler.NS(64,128,"PG")
NUS = pr.convertors.US_to_NUS(Ref,ind_NUS)
CS_NUS = pr.CS.NUS_US(NUS,ind_NUS,200)

In [ ]:
%%time
contours = {"Ubi.ft2": [0.03,1.2,20] ,"Azurin.ft2": [0.02,1.2,20],"Tau.ft2": [0.04,1.2,20],
            "Malt.ft2": [0.025,1.2,20],"Data.ft2": [0.02,1.2,20]}
lims = {"Ubi.ft2": [9.9,6,130,100],"Azurin.ft2": [10,5.6,135,100],"Tau.ft2": [8.5, 7.6,130, 105],
        "Malt.ft2": [10, 6.5,136, 100],"Data.ft2": [12.7,-3.3,138,98]}

fig, ax = plt.subplots(3,2,figsize=(2*10,3*10))
cmap = [plt.cm.Greys_r,plt.cm.Reds_r]    #[+,-] contour map (colors to use for contours)
contour = contours.get(spectrum)         #[contour level start value, scaling factor between contour level]
lim = lims.get(spectrum)                 #[xlim from,xlim to,ylim from,ylim to]

pr.PLOTS.subp(ax[0,0],dic,Ref/np.max(Ref), "Ref",contour,cmap,lim)
pr.PLOTS.subp(ax[0,1],dic,CS_NUS/np.max(Ref), "CS-NUS",contour,cmap,lim)

pr.PLOTS.subp(ax[1,0],dic,WNN_Echo/np.max(Ref), "Ai-Echo",contour,cmap,lim)
pr.PLOTS.subp(ax[1,1],dic,CS_Echo/np.max(Ref), "CS-Echo",contour,cmap,lim)

pr.PLOTS.subp(ax[2,0],dic,WNN_Anti/np.max(Ref), "Ai-Anti",contour,cmap,lim)
pr.PLOTS.subp(ax[2,1],dic,CS_Anti/np.max(Ref), "CS-Anti",contour,cmap,lim)

ax[0,0].axes.xaxis.set_visible(False)
ax[0,1].axes.xaxis.set_visible(False)
ax[0,1].axes.yaxis.set_visible(False)
ax[1,0].axes.xaxis.set_visible(False)
ax[1,1].axes.xaxis.set_visible(False)
ax[1,1].axes.yaxis.set_visible(False)
ax[2,1].axes.yaxis.set_visible(False)

plt.subplots_adjust(wspace=0.05, hspace=0.05)